<a href="https://colab.research.google.com/github/zanzivyr/Optimizers/blob/main/AlgebraicRiccatiEquation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Continuous Algebraic Riccati Equation (C.A.R.E.)

This is important for Control Theory. Solving this allows us to find an optimal closed-loop control gain for a system using LQR.



## Conditions

1. Given an affine linear state-space system:
  - ẋ = Ax + Bu
  - y = Cx + Du
  - u = -Kx (Closed-loop control law)
2. The system must be Controllable and Observable.
  - Ctrb if rank([ B AB A^2B ... A^(n-1)B ]) = n
  - Obsrv if rank([ C CA CA^2 ... CA^(n-1) ]ᵀ) = n
3. Quadratic Programming 
  - min u(.) ∫(0->inf) xᵀQx + uᵀRu dt
  - subject to ẋ = Ax + Bu and x(0) = x_init
  - (*Important*: This is an infinite horizon problem)
4. Q is positive semidefinite, R is positive definite

If these conditions are true then:

There exists an unique P (positive definite) which solves the C.A.R.E. and gives rise to gain K.

## Procedure

Using the Hamiltonian or Schur Method, execute the following:

1. Create an Hamiltonian matrix
2. Solve the eigenvalue problem, giving U
3. Check that u11 can be inverted
4. Check that P = u21*inv(u11) is positive definite
5. Multiply

# Resources

- Hamiltonian Matrix Method, Adwait Datar - https://www.youtube.com/watch?v=PKjl0kCMJ0g&t=637s
- Hamiltonian Matrix, Wikipedia - https://en.wikipedia.org/wiki/Hamiltonian_matrix
- Schur Method - https://math.stackexchange.com/questions/2504408/solving-continuous-algebraic-riccati-equation-using-generalized-eigenproblem-alg
- Generalized  Eigenproblem  Algorithms and  Software  for  Algebraic  Riccati Equations - https://engineering.purdue.edu/AAECourses/aae564/2008/fall/Notes/ArnoldLaub1984
- Tests for Positive Definiteness of a Matrix - https://www.gaussianwaves.com/2013/04/tests-for-positive-definiteness-of-a-matrix/
- Find out if matrix is positive definite with numpy - https://stackoverflow.com/questions/16266720/find-out-if-matrix-is-positive-definite-with-numpy

In [97]:
import numpy as np
from numpy import linalg as LA
import tensorflow as tf

# Instantiate

In [106]:
n = 2
size = 10

A = tf.constant([1.,2.,3.,4.], shape=[n,n])
B = tf.constant([1.,0.], shape=[n,1])
C = tf.constant([1.,0.], shape=[1,n])
Q = np.dot(np.transpose(C), C)
R = np.eye(1)

A,B,C,Q,R

(<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
 array([[1., 2.],
        [3., 4.]], dtype=float32)>,
 <tf.Tensor: shape=(2, 1), dtype=float32, numpy=
 array([[1.],
        [0.]], dtype=float32)>,
 <tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[1., 0.]], dtype=float32)>,
 array([[1., 0.],
        [0., 0.]], dtype=float32),
 array([[1.]]))

## Check Controllability and Observability

We can see that the rank of the controllability and observability matrices equals n, so they pass these checks.

In [125]:
ctrb = np.array([B, np.dot(A,B)])
obsr = np.transpose(np.array([C, np.dot(C,A)]))
n, LA.matrix_rank(ctrb.reshape((n,n))), LA.matrix_rank(obsr.reshape(n,n))

(2, 2, 2)

# 1. Create Hamiltonian Matrix

In [99]:
H11 = A
H12 = -np.dot(B, np.dot(LA.inv(R), np.transpose(B)))
H21 = -Q
H22 = -np.transpose(A)

H = np.array([H11, H12, H21, H22]).reshape(2*n, 2*n)
H

array([[ 1.,  2.,  3.,  4.],
       [-1., -0., -0., -0.],
       [-1., -0., -0., -0.],
       [-1., -3., -2., -4.]])

# 2. Solve the Eigenvalue Problem

Solve the Eigenvalue Problem using Schur Decomposition.

In [100]:
w, vset = LA.eig(H)
vectors = vset.shape[0]
size = vset.shape[1]
vset

array([[ 6.45497224e-01+0.00000000e+00j,  6.45497224e-01-0.00000000e+00j,
        -3.20418736e-17+8.04819731e-09j, -3.20418736e-17-8.04819731e-09j],
       [ 1.93649167e-01+2.14087210e-01j,  1.93649167e-01-2.14087210e-01j,
        -5.29812943e-01-7.64578745e-09j, -5.29812943e-01+7.64578745e-09j],
       [ 1.93649167e-01+2.14087210e-01j,  1.93649167e-01-2.14087210e-01j,
        -5.29812943e-01+2.41445919e-09j, -5.29812943e-01-2.41445919e-09j],
       [-6.45497224e-01+1.82560691e-17j, -6.45497224e-01-1.82560691e-17j,
         6.62266179e-01+0.00000000e+00j,  6.62266179e-01-0.00000000e+00j]])

In [101]:
basis = vset[0]

for v in range(1, vectors):
  vec = vset[v]

  sum = 0
  for u in range(1, v+1):
    uvec = vset[u]
    sum += np.dot((np.dot(uvec, vec) / np.dot(uvec, uvec)), uvec)

  b = vec - sum
  basis = tf.concat([basis, b], 0)

basis = tf.reshape(basis, (vectors,size), name=None)
basis

<tf.Tensor: shape=(4, 4), dtype=complex128, numpy=
array([[ 6.45497224e-01+0.00000000e+00j,  6.45497224e-01-0.00000000e+00j,
        -3.20418736e-17+8.04819731e-09j, -3.20418736e-17-8.04819731e-09j],
       [ 0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j],
       [-1.93649167e-01-2.14087210e-01j, -1.93649167e-01+2.14087210e-01j,
         5.29812943e-01+7.64578745e-09j,  5.29812943e-01-7.64578745e-09j],
       [ 6.76680665e-01+7.48098623e-01j,  6.76680665e-01-7.48098623e-01j,
        -1.85135924e+00-9.14008237e-09j, -1.85135924e+00+9.14008237e-09j]])>

In [102]:
ortho = 0

if LA.norm(basis[0]) != 0:
  ortho = tf.math.multiply(basis[0], 1/LA.norm(basis[0]))
else:
  ortho = tf.zeros(size, dtype=tf.complex128)

for v in range(1, vectors):
  if LA.norm(basis[v]) != 0:
    o = tf.math.multiply(basis[v], 1/LA.norm(basis[v]))
  else:
    o = tf.zeros(size, dtype=tf.complex128)
    
  ortho = tf.concat([ortho, o], 0)

ortho = tf.reshape(ortho, (vectors,size), name=None)
ortho

<tf.Tensor: shape=(4, 4), dtype=complex128, numpy=
array([[ 7.07106781e-01+0.00000000e+00j,  7.07106781e-01+0.00000000e+00j,
        -3.51001139e-17+8.81635843e-09j, -3.51001139e-17-8.81635843e-09j],
       [ 0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
         0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j],
       [-2.26949387e-01-2.50901987e-01j, -2.26949387e-01+2.50901987e-01j,
         6.20920421e-01+8.96056924e-09j,  6.20920421e-01-8.96056924e-09j],
       [ 2.26949387e-01+2.50901987e-01j,  2.26949387e-01-2.50901987e-01j,
        -6.20920421e-01-3.06545790e-09j, -6.20920421e-01+3.06545790e-09j]])>

# 3. Check that u11 can be inverted

In [107]:
U = ortho

try:
  u11inv = LA.inv(tf.reshape(U[0], [n,n]))
  u21 = tf.reshape(U[2], [n,n])
  P = np.dot(u21, u11inv)

  P
except:
  print("Infeasible! U11 is not invertible")

# 4. Check that P is positive definite

If so, multiply!

In [109]:
# Check that P is positive definite
if np.all(np.linalg.eigvals(P) > 0):
  Kstar = -np.dot(LA.inv(R), np.dot(np.transpose(B), P))

  print("Optimal gain K_star is: " + str(K))

else:
  print("P is not positive definite, therefore there is no optimal gain.")

Optimal gain K_star is: [[3.20954902e-01+7.39143466e-18j 2.84586873e+07-9.08498292e-10j]]


## K⋆ is not necessarily optimal

In this example, K⋆ is not necessarily optimal because we did not solve the QP. U will need to be stepped towards the optimal value using Newton's Method. 

However, as we saw in the previous study on QP's, Automatic Differentiation is required to complete the QP.

In the future I may attempt Automatic Differentiation, but for now we will conclude this study.